In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.metrics import classification_report, confusion_matrix
from scipy.sparse import hstack
import joblib
import os
import requests
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import string
import nltk
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt', quiet=True)
import joblib
import sklearn
from sklearn.preprocessing import StandardScaler


In [ ]:

# Load the dataset
df = pd.read_csv('/content/drive/MyDrive/labeled_data.csv')

# Display the first few rows of the dataset to understand its structure
print(df.head())

# Split the dataset into features (X) and target (y)
X = df['text']
y = df['class']  # Assuming 'class' column contains the labels 0, 1, 2 for hate speech, offensive language, neither

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize NLTK resources for text preprocessing
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# Function to preprocess text data
def preprocess_text(text):
    tokens = word_tokenize(text.lower())  # Tokenization
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and token not in string.punctuation]  # Remove stopwords and punctuation, and lemmatize
    return ' '.join(tokens)

# Apply preprocessing to the training and testing sets
X_train_processed = X_train.apply(preprocess_text)
X_test_processed = X_test.apply(preprocess_text)


In [ ]:
# Initialize TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=1004)  # Adjust max_features as needed

# Fit vectorizer and transform training data
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train_processed)

# Transform testing data (using the same vectorizer fitted on training data)
X_test_tfidf = tfidf_vectorizer.transform(X_test_processed)



In [ ]:
# Initialize Logistic Regression model
lr_classifier = LogisticRegression(max_iter=1000)
model = LogisticRegression(max_iter=1000)

# Train the model
lr_classifier.fit(X_train_tfidf, y_train)

# Predictions on the testing set
y_pred = lr_classifier.predict(X_test_tfidf)

# Evaluate model performance
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')



In [ ]:
def predict_class(comment, model, vectorizer):
    # Preprocess the comment
    processed_comment = preprocess_text(comment)
    # Vectorize the comment
    vectorized_comment = vectorizer.transform([processed_comment])
    # Predict using the model
    prediction = model.predict(vectorized_comment)
    return prediction[0]


In [ ]:

model_file = '/content/drive/MyDrive/logistic_regression_model.pkl'
vectorizer_file = '/content/drive/MyDrive/tfidf_vectorizer.pkl'

# Save the model and vectorizer
joblib.dump(lr_classifier, '/content/drive/MyDrive/model.joblib')
joblib.dump(tfidf_vectorizer, '/content/drive/MyDrive/tfidf_vectorizer.pkl')

# Assuming X_train_tfidf is your sparse training data
scaler = StandardScaler(with_mean=False)
scaler.fit(X_train_tfidf)

# Save the scaler
joblib.dump(scaler, '/content/drive/MyDrive/ scaler.joblib')
